# Toffoli

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma
from typing import *
import numpy as np
import sympy
import cirq

## `Toffoli`
The Toffoli gate.

This will flip the target bit if both controls are active. It can be thought of as
a reversible AND gate.

Like `TGate`, this is a common compilation target. The Clifford+Toffoli gateset is
universal.

#### References
[Novel constructions for the fault-tolerant Toffoli gate](https://arxiv.org/abs/1212.5069).
Cody Jones. 2012. Provides a decomposition into 4 `TGate`.


In [ ]:
from qualtran.bloqs.basic_gates import Toffoli

### Example Instances

In [ ]:
toffoli = Toffoli()

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([toffoli],
           ['`toffoli`'])

In [ ]:
show_bloq(toffoli, 'musical_score')

### Call Graph

In [ ]:
toffoli_g, toffoli_sigma = toffoli.call_graph()
show_call_graph(toffoli_g)
show_counts_sigma(toffoli_sigma)